In [23]:
import os
import sys
import random
import math
from pathlib import Path
from typing import List

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TABLEBANK_ROOT = Path(
    r"C:\Users\ahmed\Dropbox\PC\Desktop\Ahmed Sajid\Office - NCV\NCV - HTR\TableBank\Recognition"
)
IMAGES_DIR = TABLEBANK_ROOT / "Images"
ANNOT_DIR = TABLEBANK_ROOT / "Annotations"
SRC_FILE = ANNOT_DIR / "src-all_train.txt"
TGT_FILE = ANNOT_DIR / "tgt-all_train.txt"
CASCADE_ROOT = Path(
    r"C:\Users\ahmed\Dropbox\PC\Desktop\Ahmed Sajid\Office - NCV\NCV - HTR\CascadeTabNet"
)
SUBSET_SIZE = 256
BATCH_SIZE = 8
IMG_H = 64
IMG_W = 512
EMBED_DIM = 512
NUM_EPOCHS = 8
LEARNING_RATE = 1e-4
MAX_SEQ_LEN = 512
MODEL_SAVE_PATH = Path.cwd() / "tsr_castabnet_tablebank.pth"
SEED = 42

random.seed(SEED)
torch.manual_seed(SEED)


def resolve_image_path(entry: str, images_dir: Path) -> Path:
    p = Path(entry.strip())
    if p.is_absolute() and p.exists():
        return p
    candidate = images_dir / entry.strip()
    if candidate.exists():
        return candidate
    stripped = entry.strip().lstrip("./")
    candidate = images_dir / stripped
    if candidate.exists():
        return candidate
    parts = entry.strip().split("/")
    candidate = images_dir / parts[-1]
    if candidate.exists():
        return candidate
    raise FileNotFoundError(entry)


class TableBankDataset(Dataset):
    def __init__(self, src_path: Path, tgt_path: Path, images_dir: Path, max_samples: int = None):
        with open(src_path, "r", encoding="utf-8") as f:
            srcs = [l.rstrip("\n") for l in f if l.strip()]
        with open(tgt_path, "r", encoding="utf-8") as f:
            tgts = [l.rstrip("\n") for l in f if l.strip()]
        pairs = []
        for s, t in zip(srcs, tgts):
            try:
                img_path = resolve_image_path(s, images_dir)
            except FileNotFoundError:
                continue
            pairs.append((str(img_path), t))
        if max_samples:
            pairs = pairs[: max_samples]
        self.pairs = pairs
        self.transform = transforms.Compose(
            [
                transforms.Resize((IMG_H, IMG_W)),
                transforms.Grayscale(num_output_channels=3),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )
        chars = set()
        for _, tgt in pairs:
            chars.update(list(tgt))
        self.vocab = ["<pad>", "<bos>", "<eos>", "<unk>"] + sorted(chars)
        self.stoi = {c: i for i, c in enumerate(self.vocab)}
        self.itos = self.vocab

    def encode_target(self, text: str) -> List[int]:
        seq = [self.stoi["<bos>"]]
        for ch in text:
            seq.append(self.stoi.get(ch, self.stoi["<unk>"]))
            if len(seq) >= MAX_SEQ_LEN - 1:
                break
        seq.append(self.stoi["<eos>"])
        return seq

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx: int):
        img_path, tgt = self.pairs[idx]
        img = Image.open(img_path).convert("RGB")
        img = self.transform(img)
        tgt_ids = self.encode_target(tgt)
        tgt_tensor = torch.tensor(tgt_ids, dtype=torch.long)
        return img, tgt_tensor, tgt, os.path.basename(img_path)


def collate_fn(batch):
    imgs, tgts, raw, names = zip(*batch)
    imgs = torch.stack(imgs)
    lengths = [t.shape[0] for t in tgts]
    max_len = max(lengths)
    padded = torch.full((len(tgts), max_len), 0, dtype=torch.long)
    for i, t in enumerate(tgts):
        padded[i, : t.shape[0]] = t
    return imgs, padded, lengths, raw, names


def try_add_cascade_path(root: Path):
    sys.path.insert(0, str(root))
    sys.path.insert(0, str(root / "Table Structure Recognition"))


try_add_cascade_path(CASCADE_ROOT)
CASCADE_AVAILABLE = False
try:
    from CascadeTabNet.model import CascadeTabNet  # type: ignore
    CASCADE_AVAILABLE = True
except Exception:
    CASCADE_AVAILABLE = False


class EncoderWrapper(nn.Module):
    def __init__(self, out_dim: int):
        super().__init__()
        r = models.resnet18(pretrained=True)
        self.encoder = nn.Sequential(*list(r.children())[:-2])
        self.proj = nn.Linear(512, out_dim)

    def forward(self, x):
        f = self.encoder(x)
        b, c, h, w = f.size()
        f = f.view(b, c, -1).permute(0, 2, 1)
        f = self.proj(f)
        f = f.permute(1, 0, 2)
        return f


class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=2000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        if d_model % 2 == 1:
            pe[:, 1::2] = torch.cos(pos * div[:-1])
        else:
            pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer("pe", pe.unsqueeze(1))

    def forward(self, x):
        return x + self.pe[: x.size(0)]


def generate_mask(sz: int):
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float("-inf")).masked_fill(mask == 1, float(0.0))
    return mask


class Seq2SeqModel(nn.Module):
    def __init__(self, vocab_size: int, embed_dim: int):
        super().__init__()
        self.encoder = EncoderWrapper(embed_dim)
        self.token_emb = nn.Embedding(vocab_size, embed_dim)
        self.pos_enc = PositionalEncoding(embed_dim)
        decoder_layer = nn.TransformerDecoderLayer(d_model=embed_dim, nhead=8)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=3)
        self.output_fc = nn.Linear(embed_dim, vocab_size)
        self.embed_dim = embed_dim

    def forward(self, imgs, tgt_seq):
        memory = self.encoder(imgs)
        tgt_emb = self.token_emb(tgt_seq)
        tgt_emb = tgt_emb * math.sqrt(self.embed_dim)
        tgt_emb = tgt_emb.permute(1, 0, 2)
        tgt_emb = self.pos_enc(tgt_emb)
        tgt_mask = generate_mask(tgt_emb.size(0)).to(tgt_emb.device)
        out = self.decoder(tgt_emb, memory, tgt_mask=tgt_mask)
        logits = self.output_fc(out)
        return logits.permute(1, 0, 2)


def train_model():
    dataset = TableBankDataset(SRC_FILE, TGT_FILE, IMAGES_DIR, max_samples=SUBSET_SIZE)
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn)
    model = Seq2SeqModel(len(dataset.vocab), EMBED_DIM).to(DEVICE)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    for epoch in range(1, NUM_EPOCHS + 1):
        model.train()
        loop = tqdm(loader, desc=f"Epoch {epoch}", leave=True)
        total_loss = 0.0
        for imgs, tgt_padded, _, _, _ in loop:
            imgs = imgs.to(DEVICE)
            tgt_in = tgt_padded[:, :-1].to(DEVICE)
            tgt_out = tgt_padded[:, 1:].to(DEVICE)
            optimizer.zero_grad()
            logits = model(imgs, tgt_in)
            loss = criterion(logits.reshape(-1, logits.size(-1)), tgt_out.reshape(-1))
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 5.0)
            optimizer.step()
            total_loss += loss.item()
            loop.set_postfix(loss=total_loss / (loop.n + 1))
    torch.save({"model_state_dict": model.state_dict(), "vocab": dataset.vocab}, MODEL_SAVE_PATH)
    return model, dataset


def greedy_decode(model, img_tensor, stoi, itos, max_len=MAX_SEQ_LEN):
    model.eval()
    bos = stoi["<bos>"]
    eos = stoi["<eos>"]
    ys = torch.tensor([[bos]], device=DEVICE, dtype=torch.long)
    img_tensor = img_tensor.unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        memory = model.encoder(img_tensor)
        for _ in range(max_len):
            emb = model.token_emb(ys)
            emb = emb * math.sqrt(model.embed_dim)
            emb_t = emb.permute(1, 0, 2)
            emb_t = model.pos_enc(emb_t)
            tgt_mask = generate_mask(emb_t.size(0)).to(DEVICE)
            out = model.decoder(emb_t, memory, tgt_mask=tgt_mask)
            last = out[-1, 0, :]
            logits = model.output_fc(last)
            next_tok = logits.argmax(dim=-1)
            next_tok_tensor = next_tok.unsqueeze(0).unsqueeze(0)
            ys = torch.cat([ys, next_tok_tensor.to(DEVICE)], dim=1)
            if next_tok.item() == eos:
                break
    tokens = ys.squeeze(0).tolist()
    tokens = [t for t in tokens if t not in (bos, eos, 0)]
    text = "".join([itos[t] if t < len(itos) else "<unk>" for t in tokens])
    return text


def quick_test(model, dataset, n=5):
    tf = transforms.Compose(
        [
            transforms.Resize((IMG_H, IMG_W)),
            transforms.Grayscale(num_output_channels=3),
            transforms.ToTensor(),
            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
        ]
    )
    sample_ids = random.sample(range(len(dataset)), min(n, len(dataset)))
    for i in sample_ids:
        img_path, tgt = dataset.pairs[i]
        img = Image.open(img_path).convert("RGB")
        img_t = tf(img)
        pred = greedy_decode(model, img_t, dataset.stoi, dataset.itos)
        print("IMAGE:", os.path.basename(img_path))
        print("PRED :", pred)
        print("GT   :", tgt)
        print("-" * 80)


if __name__ == "__main__":
    model, ds = train_model()
    quick_test(model, ds, n=8)


Epoch 8: 100%|█████████████████████████████████████████████████████████████| 32/32 [00:03<00:00,  9.66it/s, loss=0.217]


RuntimeError: shape '[32, 16, 64]' is invalid for input of size 16384

In [5]:
import os
import sys
import re
import random
import math
from pathlib import Path
from typing import List, Tuple, Dict

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ROOT = Path(r"C:\Users\ahmed\Dropbox\PC\Desktop\Ahmed Sajid\Office - NCV\NCV - HTR\TableBank\Recognition")
IMAGES_DIR = ROOT / "Images"
ANNOT_DIR = ROOT / "Annotations"
SRC_FILE = ANNOT_DIR / "src-all_train.txt"
TGT_FILE = ANNOT_DIR / "tgt-all_train.txt"

SUBSET_SIZE = 1024
BATCH_SIZE = 8
IMG_H = 64
IMG_W = 512
EMBED_DIM = 256
ENCODER_DIM = 512
DEC_HID = 512
NUM_EPOCHS = 6
LR = 3e-4
MAX_LEN = 150
COVERAGE_LAMBDA = 0.4
MODEL_SAVE = Path.cwd() / "final_tsr_model.pth"

random.seed(42)
torch.manual_seed(42)


def resolve_path(s, root):
    s = s.strip()
    p = Path(s)
    if p.exists():
        return p
    t = root / s
    if t.exists():
        return t
    s2 = s.replace("\\", "/").split("/")[-1]
    t = root / s2
    if t.exists():
        return t
    raise FileNotFoundError(s)


def tokenize_html(t):
    toks = re.findall(r"</?[^>]+>|[^<>]", t)
    return toks[: MAX_LEN - 3]


class TableBankDS(Dataset):
    def __init__(self, src, tgt, root, limit=None):
        with open(src, "r", encoding="utf-8") as f:
            s = [l.strip() for l in f if l.strip()]
        with open(tgt, "r", encoding="utf-8") as f:
            g = [l.strip() for l in f if l.strip()]
        pairs = []
        for a, b in zip(s, g):
            try:
                p = resolve_path(a, root)
            except:
                continue
            toks = tokenize_html(b)
            if toks:
                pairs.append((str(p), toks))
        if limit:
            pairs = pairs[:limit]
        self.pairs = pairs
        token_counts = {}
        for _, t in pairs:
            for x in t:
                token_counts[x] = token_counts.get(x, 0) + 1
        special = ["<pad>", "<bos>", "<eos>", "<unk>", "<eot>"]
        vocab = special + sorted(token_counts.keys())
        self.itos = vocab
        self.stoi = {c: i for i, c in enumerate(vocab)}
        self.counts = [token_counts.get(t, 0) for t in vocab]
        self.vocab = len(vocab)
        self.tf = transforms.Compose(
            [
                transforms.Resize((IMG_H, IMG_W)),
                transforms.Grayscale(3),
                transforms.ToTensor(),
                transforms.Normalize([0.485,0.456,0.406],[0.229,0.224,0.225])
            ]
        )

    def encode(self, toks):
        arr = [self.stoi["<bos>"]]
        for t in toks:
            arr.append(self.stoi.get(t, self.stoi["<unk>"]))
            if len(arr) >= MAX_LEN - 2:
                break
        arr.append(self.stoi["<eos>"])
        arr.append(self.stoi["<eot>"])
        return torch.tensor(arr, dtype=torch.long)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, i):
        p, t = self.pairs[i]
        img = Image.open(p).convert("RGB")
        img = self.tf(img)
        enc = self.encode(t)
        return img, enc, " ".join(t), os.path.basename(p)


def collate(batch):
    imgs, tgts, raws, names = zip(*batch)
    imgs = torch.stack(imgs)
    L = [t.shape[0] for t in tgts]
    m = max(L)
    pad = torch.full((len(tgts), m), 0, dtype=torch.long)
    for i, t in enumerate(tgts):
        pad[i, : t.shape[0]] = t
    return imgs, pad, raws, names


class Encoder(nn.Module):
    def __init__(self, dim):
        super().__init__()
        r = models.resnet18(pretrained=True)
        self.f = nn.Sequential(*list(r.children())[:-2])
        self.p = nn.Linear(512, dim)

    def forward(self, x):
        z = self.f(x)
        b, c, h, w = z.shape
        z = z.view(b, c, -1).permute(0,2,1)
        z = self.p(z)
        return z.permute(1,0,2)


class CovAttention(nn.Module):
    def __init__(self, enc_dim, dec_dim):
        super().__init__()
        self.W_enc = nn.Linear(enc_dim, dec_dim, bias=False)
        self.W_dec = nn.Linear(dec_dim, dec_dim, bias=False)
        self.W_cov = nn.Linear(1, dec_dim, bias=False)
        self.v = nn.Linear(dec_dim,1,bias=False)

    def forward(self, enc, h, cov):
        seq, b, ed = enc.size()
        e = enc.permute(1,0,2)
        enc_proj = self.W_enc(e)
        dec_proj = self.W_dec(h).unsqueeze(1)
        cov_proj = self.W_cov(cov.unsqueeze(2))
        s = self.v(torch.tanh(enc_proj + dec_proj + cov_proj)).squeeze(2)
        attn = torch.softmax(s, dim=1)
        ctx = torch.bmm(attn.unsqueeze(1), e).squeeze(1)
        return ctx, attn# (1, L, dec_dim)


class Decoder(nn.Module):
    def __init__(self, vocab, emb, hid, enc_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab, emb, padding_idx=0)
        self.attn = CovAttention(enc_dim, hid)
        self.lstm = nn.LSTMCell(emb + enc_dim, hid)
        self.fc = nn.Linear(hid, vocab)
        self.hid = hid

    def forward(self, enc, tgt):
        seq, b, ed = enc.size()
        cov = torch.zeros(b, seq, device=enc.device)
        h = torch.zeros(b, self.hid, device=enc.device)
        c = torch.zeros(b, self.hid, device=enc.device)
        out = []
        x = self.emb(tgt[:,0])
        for t in range(1, tgt.size(1)):
            ctx, attn = self.attn(enc, h, cov)
            inp = torch.cat([x, ctx], dim=1)
            h, c = self.lstm(inp, (h,c))
            logits = self.fc(h)
            out.append(logits.unsqueeze(1))
            cov = cov + attn.detach()
            x = self.emb(tgt[:,t])
        return torch.cat(out,1), cov


class Model(nn.Module):
    def __init__(self, vocab):
        super().__init__()
        self.enc = Encoder(ENCODER_DIM)
        self.dec = Decoder(vocab, EMBED_DIM, DEC_HID, ENCODER_DIM)

    def forward(self, imgs, tgt):
        mem = self.enc(imgs)
        out, cov = self.dec(mem, tgt)
        return out, cov


def class_w(counts):
    t = torch.tensor(counts, dtype=torch.float)
    t = t+1
    t = 1/torch.sqrt(t)
    t = t/t.mean()
    return torch.clamp(t,0.1,5).to(DEVICE)


def train():
    ds = TableBankDS(SRC_FILE, TGT_FILE, IMAGES_DIR, SUBSET_SIZE)
    dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)
    model = Model(ds.vocab).to(DEVICE)
    w = class_w(ds.counts)
    loss_f = nn.CrossEntropyLoss(weight=w, ignore_index=0)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    for e in range(NUM_EPOCHS):
        model.train()
        loop = tqdm(dl, desc=f"Epoch {e+1}")
        for imgs, tgt, _, _ in loop:
            imgs = imgs.to(DEVICE)
            tgt = tgt.to(DEVICE)
            inp = tgt[:,:-1]
            out = tgt[:,1:]
            opt.zero_grad()
            logits, cov = model(imgs, inp)
            b,t,v = logits.size()
            out = out[:,:t]
            loss_tok = loss_f(logits.reshape(-1,v), out.reshape(-1))
            loss = loss_tok + COVERAGE_LAMBDA * cov.mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(),1.0)
            opt.step()
            loop.set_postfix(loss=float(loss))
    torch.save({"state":model.state_dict(),"itos":ds.itos,"stoi":ds.stoi}, MODEL_SAVE)
    return model, ds


def decode(model, img, stoi, itos):
    model.eval()
    img = img.unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        mem = model.enc(img)
        seq,b,ed = mem.size()
        cov = torch.zeros(b, seq, device=DEVICE)
        h = torch.zeros(b, DEC_HID, device=DEVICE)
        c = torch.zeros(b, DEC_HID, device=DEVICE)
        cur = torch.tensor([stoi["<bos>"]], device=DEVICE)
        x = model.dec.emb(cur)
        out=[]
        for _ in range(MAX_LEN):
            ctx, attn = model.dec.attn(mem, h, cov)
            inp = torch.cat([x,ctx],1)
            h,c = model.dec.lstm(inp,(h,c))
            lg = model.dec.fc(h)
            nxt = lg.argmax(1)
            if int(nxt)==stoi["<eot>"]:
                break
            out.append(int(nxt))
            cov = cov + attn.detach()
            x = model.dec.emb(nxt)
        return "".join(itos[i] for i in out if i<len(itos))


def predict_all(model, ds):
    res=[]
    tf = ds.tf
    for p,toks in tqdm(ds.pairs):
        img = Image.open(p).convert("RGB")
        img = tf(img)
        pred = decode(model,img,ds.stoi,ds.itos)
        res.append((p,pred," ".join(toks)))
    return res


if __name__=="__main__":
    m,ds = train()
    preds = predict_all(m,ds)

100%|██████████████████████████████████████████████████████████████████████████████| 1024/1024 [02:28<00:00,  6.90it/s]


In [21]:
import os
import re
import random
import math
from pathlib import Path
from typing import List, Tuple, Dict

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ROOT = Path(r"C:\Users\ahmed\Dropbox\PC\Desktop\Ahmed Sajid\Office - NCV\NCV - HTR\TableBank\Recognition")
IMAGES_DIR = ROOT / "Images"
ANNOT_DIR = ROOT / "Annotations"
SRC_FILE = ANNOT_DIR / "src-all_train.txt"
TGT_FILE = ANNOT_DIR / "tgt-all_train.txt"
SUBSET_SIZE = 1024
BATCH_SIZE = 8
IMG_H = 64
IMG_W = 512
EMBED_DIM = 256
ENCODER_DIM = 512
DEC_HID = 512
NUM_EPOCHS = 6
LR = 3e-4
MAX_LEN = 150
COVERAGE_LAMBDA = 0.4
MODEL_SAVE = Path.cwd() / "final_tsr_model.pth"
random.seed(42)
torch.manual_seed(42)


def resolve_path(s: str, root: Path) -> Path:
    s = s.strip()
    p = Path(s)
    if p.exists():
        return p
    t = root / s
    if t.exists():
        return t
    s2 = s.replace("\\", "/").split("/")[-1]
    t = root / s2
    if t.exists():
        return t
    raise FileNotFoundError(s)


def tokenize_html(t: str) -> List[str]:
    toks = re.findall(r"</?[^>]+>|[^<>]", t)
    return toks[: MAX_LEN - 3]


class TableBankDS(Dataset):
    def __init__(self, src: Path, tgt: Path, root: Path, limit: int = None):
        with open(src, "r", encoding="utf-8") as f:
            s = [l.strip() for l in f if l.strip()]
        with open(tgt, "r", encoding="utf-8") as f:
            g = [l.strip() for l in f if l.strip()]
        pairs: List[Tuple[str, List[str]]] = []
        for a, b in zip(s, g):
            try:
                p = resolve_path(a, root)
            except FileNotFoundError:
                continue
            toks = tokenize_html(b)
            if toks:
                pairs.append((str(p), toks))
        if limit:
            pairs = pairs[:limit]
        self.pairs = pairs
        token_counts: Dict[str, int] = {}
        for _, tks in pairs:
            for x in tks:
                token_counts[x] = token_counts.get(x, 0) + 1
        special = ["<pad>", "<bos>", "<eos>", "<unk>", "<eot>"]
        vocab = special + sorted(token_counts.keys())
        self.itos = vocab
        self.stoi = {c: i for i, c in enumerate(vocab)}
        self.counts = [token_counts.get(t, 0) for t in vocab]
        self.vocab = len(vocab)
        self.tf = transforms.Compose(
            [
                transforms.Resize((IMG_H, IMG_W)),
                transforms.Grayscale(num_output_channels=3),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def encode(self, toks: List[str]) -> torch.Tensor:
        arr = [self.stoi["<bos>"]]
        for t in toks:
            arr.append(self.stoi.get(t, self.stoi["<unk>"]))
            if len(arr) >= MAX_LEN - 2:
                break
        arr.append(self.stoi["<eos>"])
        arr.append(self.stoi["<eot>"])
        return torch.tensor(arr, dtype=torch.long)

    def __len__(self) -> int:
        return len(self.pairs)

    def __getitem__(self, i: int):
        p, t = self.pairs[i]
        img = Image.open(p).convert("RGB")
        img = self.tf(img)
        enc = self.encode(t)
        return img, enc, " ".join(t), os.path.basename(p)


def collate(batch):
    imgs, tgts, raws, names = zip(*batch)
    imgs = torch.stack(imgs)
    L = [t.shape[0] for t in tgts]
    m = max(L)
    pad = torch.full((len(tgts), m), 0, dtype=torch.long)
    for i, t in enumerate(tgts):
        pad[i, : t.shape[0]] = t
    return imgs, pad, raws, names


class Encoder(nn.Module):
    def __init__(self, dim: int):
        super().__init__()
        r = models.resnet18(pretrained=True)
        self.f = nn.Sequential(*list(r.children())[:-2])
        self.p = nn.Linear(512, dim)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        z = self.f(x)
        b, c, h, w = z.shape
        z = z.view(b, c, -1).permute(0, 2, 1)
        z = self.p(z)
        return z.permute(1, 0, 2)


class CovAttention(nn.Module):
    def __init__(self, enc_dim: int, dec_dim: int):
        super().__init__()
        self.W_enc = nn.Linear(enc_dim, dec_dim, bias=False)
        self.W_dec = nn.Linear(dec_dim, dec_dim, bias=False)
        self.W_cov = nn.Linear(1, dec_dim, bias=False)
        self.v = nn.Linear(dec_dim, 1, bias=False)

    def forward(self, enc: torch.Tensor, h: torch.Tensor, cov: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        seq, b, ed = enc.size()
        e = enc.permute(1, 0, 2)
        enc_proj = self.W_enc(e)
        dec_proj = self.W_dec(h).unsqueeze(1)
        cov_proj = self.W_cov(cov.unsqueeze(2))
        s = self.v(torch.tanh(enc_proj + dec_proj + cov_proj)).squeeze(2)
        attn = torch.softmax(s, dim=1)
        ctx = torch.bmm(attn.unsqueeze(1), e).squeeze(1)
        return ctx, attn


class Decoder(nn.Module):
    def __init__(self, vocab: int, emb: int, hid: int, enc_dim: int):
        super().__init__()
        self.emb = nn.Embedding(vocab, emb, padding_idx=0)
        self.attn = CovAttention(enc_dim, hid)
        self.lstm = nn.LSTMCell(emb + enc_dim, hid)
        self.fc = nn.Linear(hid, vocab)
        self.hid = hid

    def forward(self, enc: torch.Tensor, tgt: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        seq, b, ed = enc.size()
        cov = torch.zeros(b, seq, device=enc.device)
        h = torch.zeros(b, self.hid, device=enc.device)
        c = torch.zeros(b, self.hid, device=enc.device)
        out = []
        x = self.emb(tgt[:, 0])
        for t in range(1, tgt.size(1)):
            ctx, attn = self.attn(enc, h, cov)
            inp = torch.cat([x, ctx], dim=1)
            h, c = self.lstm(inp, (h, c))
            logits = self.fc(h)
            out.append(logits.unsqueeze(1))
            cov = cov + attn.detach()
            x = self.emb(tgt[:, t])
        return torch.cat(out, 1), cov


class Model(nn.Module):
    def __init__(self, vocab: int):
        super().__init__()
        self.enc = Encoder(ENCODER_DIM)
        self.dec = Decoder(vocab, EMBED_DIM, DEC_HID, ENCODER_DIM)

    def forward(self, imgs: torch.Tensor, tgt: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        mem = self.enc(imgs)
        out, cov = self.dec(mem, tgt)
        return out, cov


def class_w(counts: List[int]) -> torch.Tensor:
    t = torch.tensor(counts, dtype=torch.float)
    t = t + 1.0
    t = 1.0 / torch.sqrt(t)
    t = t / t.mean()
    return torch.clamp(t, 0.1, 5.0).to(DEVICE)


def train():
    ds = TableBankDS(SRC_FILE, TGT_FILE, IMAGES_DIR, SUBSET_SIZE)
    dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)
    model = Model(ds.vocab).to(DEVICE)
    w = class_w(ds.counts)
    loss_f = nn.CrossEntropyLoss(weight=w, ignore_index=0)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    for e in range(NUM_EPOCHS):
        model.train()
        loop = tqdm(dl, desc=f"Epoch {e+1}")
        for imgs, tgt, _, _ in loop:
            imgs = imgs.to(DEVICE)
            tgt = tgt.to(DEVICE)
            inp = tgt[:, :-1]
            out = tgt[:, 1:]
            opt.zero_grad()
            logits, cov = model(imgs, inp)
            b, t, v = logits.size()
            out = out[:, :t]
            loss_tok = loss_f(logits.reshape(-1, v), out.reshape(-1))
            loss = loss_tok + COVERAGE_LAMBDA * cov.mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            loop.set_postfix(loss=float(loss))
    torch.save({"state": model.state_dict(), "itos": ds.itos, "stoi": ds.stoi}, MODEL_SAVE)
    return model, ds


def decode(model: Model, img: torch.Tensor, stoi: Dict[str, int], itos: List[str]) -> str:
    model.eval()
    img = img.unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        mem = model.enc(img)  # (T, B, enc_dim)
        seq, b, ed = mem.size()
        cov = torch.zeros(b, seq, device=DEVICE)
        h = torch.zeros(b, DEC_HID, device=DEVICE)
        c = torch.zeros(b, DEC_HID, device=DEVICE)
        cur = torch.tensor([stoi["<bos>"]], device=DEVICE)
        x = model.dec.emb(cur)
        out = []
        for _ in range(MAX_LEN):
            ctx, attn = model.dec.attn(mem, h, cov)
            inp = torch.cat([x, ctx], dim=1)
            h, c = model.dec.lstm(inp, (h, c))
            lg = model.dec.fc(h)
            nxt = lg.argmax(dim=1)
            val = int(nxt.item())
            if val == stoi["<eot>"]:
                break
            out.append(val)
            cov = cov + attn.detach()
            x = model.dec.emb(nxt)
        text = "".join(itos[i] if i < len(itos) else "<unk>" for i in out if i not in (stoi["<bos>"], 0))
        return text


def evaluate_and_print(model: Model, ds: TableBankDS, save_csv: Path = Path.cwd() / "predictions_tablebank.csv"):
    model.eval()
    tf = ds.tf
    rows = []
    for img_path, toks in tqdm(ds.pairs, desc="Evaluating", ncols=100):
        img = Image.open(img_path).convert("RGB")
        img_t = tf(img)
        pred = decode(model, img_t, ds.stoi, ds.itos)
        gt = " ".join(toks)
        print(f"IMAGE: {os.path.basename(img_path)}")
        print(f"PRED : {pred}")
        print(f"GT   : {gt}")
        print("-" * 80)
        rows.append((os.path.basename(img_path), pred, gt))
    with open(save_csv, "w", encoding="utf-8") as f:
        f.write("image,prediction,ground_truth\n")
        for img, pred, gt in rows:
            safe_pred = pred.replace(",", " ")
            safe_gt = gt.replace(",", " ")
            f.write(f"{img},{safe_pred},{safe_gt}\n")
    return rows

def resume_train(checkpoint_path: str):
    ds = TableBankDS(SRC_FILE, TGT_FILE, IMAGES_DIR, SUBSET_SIZE)
    dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)

    model = Model(ds.vocab).to(DEVICE)

    if not os.path.exists(checkpoint_path):
        raise FileNotFoundError(f"Checkpoint not found: {checkpoint_path}")

    ckpt = torch.load(checkpoint_path, map_location=DEVICE)
    model.load_state_dict(ckpt["state"])

    print(f"Loaded model: {checkpoint_path}")

    optimizer = torch.optim.AdamW(model.parameters(), lr=LR)

    if "optimizer" in ckpt:
        optimizer.load_state_dict(ckpt["optimizer"])

    start_epoch = ckpt.get("epoch", 1)
    print(f"Resuming from epoch {start_epoch}")

    w = class_w(ds.counts)
    loss_f = nn.CrossEntropyLoss(weight=w, ignore_index=0)

    for e in range(start_epoch, NUM_EPOCHS + 1):
        model.train()
        loop = tqdm(dl, desc=f"Epoch {e}/{NUM_EPOCHS}")
    
        for imgs, tgt, _, _ in loop:
            imgs, tgt = imgs.to(DEVICE), tgt.to(DEVICE)
    
            inp = tgt[:, :-1]
            out = tgt[:, 1:]
    
            optimizer.zero_grad()
    
            logits, cov = model(imgs, inp)
            b, t, v = logits.shape
            out = out[:, :t]
    
            loss_tok = loss_f(logits.reshape(-1, v), out.reshape(-1))
            loss = loss_tok + COVERAGE_LAMBDA * cov.mean()
    
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
    
            loop.set_postfix(loss=float(loss))
    
        save_path = f"resume_epoch_{e}.pth"
        torch.save({
            "state": model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "epoch": e + 1,
            "itos": ds.itos,
            "stoi": ds.stoi
        }, save_path)
    
        print(f"Saved checkpoint: {save_path}")
        
    torch.save({
        "state": model.state_dict(),
        "itos": ds.itos,
        "stoi": ds.stoi
    }, "final_tsr_model.pth")
    
    print("\n=====================================")
    print("✔ FINAL MODEL SAVED → final_tsr_model.pth")
    print("=====================================\n")
    
    return model, ds



if __name__ == "__main__":
    model, ds = resume_train("final_tsr_model.pth")
    evaluate_and_print(model, ds)

    '''
    if MODEL_SAVE.exists():
        ckpt = torch.load(MODEL_SAVE, map_location=DEVICE)
        ds = TableBankDS(SRC_FILE, TGT_FILE, IMAGES_DIR, SUBSET_SIZE)
        model = Model(ds.vocab).to(DEVICE)
        model.load_state_dict(ckpt["state"])
        evaluate_and_print(model, ds)
    else:
        model, ds = train()
        evaluate_and_print(model, ds)
    '''


C:\Users\ahmed\AppData\Local\Temp\ipykernel_35064\526843659.py:296: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(checkpoint_path, map_location=DEVICE)


Loaded model: final_tsr_model.pth
Resuming from epoch 1


Epoch 1/6: 100%|██████████████████████████████████████████████████████████| 128/128 [01:04<00:00,  1.98it/s, loss=1.93]


Saved checkpoint: resume_epoch_1.pth


Epoch 2/6: 100%|██████████████████████████████████████████████████████████| 128/128 [01:04<00:00,  1.99it/s, loss=1.91]


...

IMAGE: 1710.02064.table_6.png
PRED : <table> <thead> <tr> <td> <td> <td> <td> </tr> </thead> <tbody> <tr> <td> <td> <td> <td> </tr> <tr> <td> <td> <td> <td> </tr> </tbody> </table><eos>
GT   : <table>   <tbody>   <tr>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   </tr>   </tbody>   </table>
--------------------------------------------------------------------------------
IMAGE: 1412.6418.table_0.png
PRED : <table> <thead> <tr> <td> <td> <td> </tr> </thead> <tbody> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr><eos>
GT   : <table>   <thead>   <tr>   <td>   <td>   <td>   <td>   

Evaluating: 100%|██████████████████████████████████████████████▊| 1021/1024 [02:15<00:00,  7.26it/s]

IMAGE: 1405.3150.table_1.png
PRED : <table> <thead> <tr> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> </tr> <tr> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> </tr> <tr> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td> <td><eos><td><eos>
GT   : <table>   <thead>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   </thead>   <tbody>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   </tbody>   </table>
--------------------------------------------------------------------------------
IMAGE: 1803.10766.table_27.png
PRED : <table> <thead> <tr> <td> <td> <td> <td> <td> <t

Evaluating: 100%|███████████████████████████████████████████████| 1024/1024 [02:15<00:00,  7.56it/s]

IMAGE: 1602.01215.table_2.png
PRED : <table> <thead> <tr> <td> <td> </tr> </thead> <tbody> <tr> <td> <td> </tr> <tr> <td> <td> </tr> <tr> <td> <td> </tr> </tbody> </table><eos>
GT   : <table>   <thead>   <tr>   <td>   <td>   </tr>   </thead>   <tbody>   <tr>   <td>   <td>   </tr>   <tr>   <td>   <td>   </tr>   </tbody>   </table>
--------------------------------------------------------------------------------
IMAGE: 1505.06583.table_0.png
PRED : <table> <thead> <tr> <td> <td> <td> </tr> </thead> <tbody> <tr> <td> <td> <td> </tr> <tr> <td> <td> <td> </tr> </tbody> </table><eos>
GT   : <table>   <thead>   <tr>   <td>   <td>   <td>   </tr>   </thead>   <tbody>   <tr>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   </tr>   </tbody>   </table>
--------------------------------------------------------------------------------
IMAGE: 1411.3084.table_1.png
PRED : <table> <thead> <tr> <td> <td> </tr> </thead> <tbody> <tr> <td> <td> </tr> <tr> <td> <td> </tr> <tr> <td> <td> </tr> <tr> <

In [3]:
import os
import re
import random
from pathlib import Path
from typing import List, Tuple

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm
from apted import APTED, Config as APTEDConfig
from apted.helpers import Tree

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ROOT = Path(
    r"C:\Users\ahmed\Dropbox\PC\Desktop\Ahmed Sajid\Office - NCV\NCV - HTR\TableBank\Recognition"
)
IMAGES_DIR = ROOT / "Images"
ANNOT_DIR = ROOT / "Annotations"
SRC_FILE = ANNOT_DIR / "src-all_train.txt"
TGT_FILE = ANNOT_DIR / "tgt-all_train.txt"

SUBSET_SIZE = 512
BATCH_SIZE = 8
IMG_H = 64
IMG_W = 512
EMBED_DIM = 256
ENCODER_DIM = 512
DEC_HID = 512
EPOCHS_PER_RUN = 6
LR = 3e-4
MAX_LEN = 250
COVERAGE_LAMBDA = 0.4
MODEL_SAVE = Path.cwd() / "final_tsr_model.pth"

random.seed(42)
torch.manual_seed(42)


def resolve_path(s, root):
    s = s.strip()
    p = Path(s)
    if p.exists():
        return p
    t = root / s
    if t.exists():
        return t
    s2 = s.replace("\\", "/").split("/")[-1]
    t = root / s2
    if t.exists():
        return t
    raise FileNotFoundError(s)


def tokenize_html(t):
    toks = re.findall(r"</?[^>]+>|[^<>]", t)
    return toks[: MAX_LEN - 3]


class TableBankDS(Dataset):
    def __init__(self, src, tgt, root, limit=None, itos=None, stoi=None):
        with open(src, "r", encoding="utf-8") as f:
            s = [l.strip() for l in f if l.strip()]
        with open(tgt, "r", encoding="utf-8") as f:
            g = [l.strip() for l in f if l.strip()]
        pairs = []
        for a, b in zip(s, g):
            try:
                p = resolve_path(a, root)
            except:
                continue
            toks = tokenize_html(b)
            if toks:
                pairs.append((str(p), toks))
        if limit:
            pairs = pairs[:limit]
        self.pairs = pairs
        if itos is not None and stoi is not None:
            self.itos = itos
            self.stoi = stoi
            self.vocab = len(itos)
            token_counts = {}
            for _, t in pairs:
                for x in t:
                    token_counts[x] = token_counts.get(x, 0) + 1
            self.counts = [token_counts.get(t, 0) for t in itos]
        else:
            token_counts = {}
            for _, t in pairs:
                for x in t:
                    token_counts[x] = token_counts.get(x, 0) + 1
            special = ["<pad>", "<bos>", "<eos>", "<unk>", "<eot>"]
            vocab = special + sorted(token_counts.keys())
            self.itos = vocab
            self.stoi = {c: i for i, c in enumerate(vocab)}
            self.counts = [token_counts.get(t, 0) for t in vocab]
            self.vocab = len(vocab)
        self.tf = transforms.Compose(
            [
                transforms.Resize((IMG_H, IMG_W)),
                transforms.Grayscale(3),
                transforms.ToTensor(),
                transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
            ]
        )

    def encode(self, toks):
        arr = [self.stoi["<bos>"]]
        for t in toks:
            arr.append(self.stoi.get(t, self.stoi["<unk>"]))
            if len(arr) >= MAX_LEN - 2:
                break
        arr.append(self.stoi["<eos>"])
        arr.append(self.stoi["<eot>"])
        return torch.tensor(arr, dtype=torch.long)

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, i):
        p, t = self.pairs[i]
        img = Image.open(p).convert("RGB")
        img = self.tf(img)
        enc = self.encode(t)
        return img, enc, " ".join(t), os.path.basename(p)


def collate(batch):
    imgs, tgts, raws, names = zip(*batch)
    imgs = torch.stack(imgs)
    L = [t.shape[0] for t in tgts]
    m = max(L)
    pad = torch.full((len(tgts), m), 0, dtype=torch.long)
    for i, t in enumerate(tgts):
        pad[i, : t.shape[0]] = t
    return imgs, pad, raws, names


class Encoder(nn.Module):
    def __init__(self, dim):
        super().__init__()
        r = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
        self.f = nn.Sequential(*list(r.children())[:-2])
        self.p = nn.Linear(512, dim)

    def forward(self, x):
        z = self.f(x)
        b, c, h, w = z.shape
        z = z.view(b, c, -1).permute(0, 2, 1)
        z = self.p(z)
        return z.permute(1, 0, 2)


class CovAttention(nn.Module):
    def __init__(self, enc_dim, dec_dim):
        super().__init__()
        self.W_enc = nn.Linear(enc_dim, dec_dim, bias=False)
        self.W_dec = nn.Linear(dec_dim, dec_dim, bias=False)
        self.W_cov = nn.Linear(1, dec_dim, bias=False)
        self.v = nn.Linear(dec_dim, 1, bias=False)

    def forward(self, enc, h, cov):
        seq, b, ed = enc.size()
        e = enc.permute(1, 0, 2)
        enc_proj = self.W_enc(e)
        dec_proj = self.W_dec(h).unsqueeze(1)
        cov_proj = self.W_cov(cov.unsqueeze(2))
        s = self.v(torch.tanh(enc_proj + dec_proj + cov_proj)).squeeze(2)
        attn = torch.softmax(s, dim=1)
        ctx = torch.bmm(attn.unsqueeze(1), e).squeeze(1)
        return ctx, attn


class Decoder(nn.Module):
    def __init__(self, vocab, emb, hid, enc_dim):
        super().__init__()
        self.emb = nn.Embedding(vocab, emb, padding_idx=0)
        self.attn = CovAttention(enc_dim, hid)
        self.lstm = nn.LSTMCell(emb + enc_dim, hid)
        self.fc = nn.Linear(hid, vocab)
        self.hid = hid

    def forward(self, enc, tgt):
        seq, b, ed = enc.size()
        cov = torch.zeros(b, seq, device=DEVICE)
        h = torch.zeros(b, self.hid, device=DEVICE)
        c = torch.zeros(b, self.hid, device=DEVICE)
        out = []
        x = self.emb(tgt[:, 0])
        for t in range(1, tgt.size(1)):
            ctx, attn = self.attn(enc, h, cov)
            inp = torch.cat([x, ctx], dim=1)
            h, c = self.lstm(inp, (h, c))
            logits = self.fc(h)
            out.append(logits.unsqueeze(1))
            cov = cov + attn.detach()
            x = self.emb(tgt[:, t])
        return torch.cat(out, 1), cov


class Model(nn.Module):
    def __init__(self, vocab):
        super().__init__()
        self.enc = Encoder(ENCODER_DIM)
        self.dec = Decoder(vocab, EMBED_DIM, DEC_HID, ENCODER_DIM)

    def forward(self, imgs, tgt):
        mem = self.enc(imgs)
        out, cov = self.dec(mem, tgt)
        return out, cov


def class_w(counts):
    t = torch.tensor(counts, dtype=torch.float)
    t = t + 1
    t = 1 / torch.sqrt(t)
    t = t / t.mean()
    return torch.clamp(t, 0.1, 5).to(DEVICE)


def train(ds, model=None, resume=False):
    dl = DataLoader(ds, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate)
    if model is None:
        model = Model(ds.vocab).to(DEVICE)
    w = class_w(ds.counts)
    loss_f = nn.CrossEntropyLoss(weight=w, ignore_index=0)
    opt = torch.optim.AdamW(model.parameters(), lr=LR)
    start_epoch = 0
    if resume and MODEL_SAVE.exists():
        ckpt = torch.load(MODEL_SAVE, map_location=DEVICE)
        model.load_state_dict(ckpt["state"])
        start_epoch = ckpt.get("epoch", 0)
    end_epoch = start_epoch + EPOCHS_PER_RUN
    for e in range(start_epoch, end_epoch):
        model.train()
        loop = tqdm(dl, desc=f"Epoch {e + 1}")
        for imgs, tgt, _, _ in loop:
            imgs = imgs.to(DEVICE)
            tgt = tgt.to(DEVICE)
            inp = tgt[:, :-1]
            out = tgt[:, 1:]
            opt.zero_grad()
            logits, cov = model(imgs, inp)
            b, t, v = logits.size()
            out = out[:, :t]
            loss_tok = loss_f(logits.reshape(-1, v), out.reshape(-1))
            loss = loss_tok + COVERAGE_LAMBDA * cov.mean()
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            opt.step()
            loop.set_postfix(loss=float(loss))
    torch.save(
        {"state": model.state_dict(), "itos": ds.itos, "stoi": ds.stoi, "epoch": end_epoch},
        MODEL_SAVE,
    )
    return model, ds


def decode_beam(model, img, stoi, itos, beam_size=3):
    model.eval()
    img = img.unsqueeze(0).to(DEVICE)
    with torch.no_grad():
        mem = model.enc(img)
        seq, b, ed = mem.size()
        cov = torch.zeros(b, seq, device=DEVICE)
        h = torch.zeros(b, DEC_HID, device=DEVICE)
        c = torch.zeros(b, DEC_HID, device=DEVICE)
        beams = [(0, [stoi["<bos>"]], h, c, cov)]
        for _ in range(MAX_LEN):
            new_beams = []
            for score, seq_ids, h, c, cov in beams:
                x = model.dec.emb(torch.tensor([seq_ids[-1]], device=DEVICE))
                ctx, attn = model.dec.attn(mem, h, cov)
                inp = torch.cat([x, ctx], 1)
                h, c = model.dec.lstm(inp, (h, c))
                lg = model.dec.fc(h)
                probs = torch.softmax(lg, dim=1).squeeze(0)
                topk = torch.topk(probs, beam_size)
                for i in range(beam_size):
                    nxt = topk.indices[i].item()
                    nscore = score - torch.log(topk.values[i] + 1e-9).item()
                    new_beams.append((nscore, seq_ids + [nxt], h, c, cov + attn.detach()))
            new_beams = sorted(new_beams, key=lambda x: x[0])[:beam_size]
            beams = new_beams
        best_seq = beams[0][1]
        return "".join(
            itos[i] for i in best_seq if i < len(itos) and itos[i] not in ["<bos>", "<eos>", "<eot>", "<pad>"]
        )


def teds_score(pred, gt):
    def parse(s):
        tokens = re.findall(r"</?[^>]+>|[^<>]+", s)
        stack = []
        root = Tree("root")
        stack.append(root)
        i = 0
        while i < len(tokens):
            tok = tokens[i]
            if tok.startswith("</"):
                if len(stack) > 1:
                    stack.pop()
            elif tok.startswith("<"):
                node = Tree(tok)
                stack[-1].add_child(node)
                if not tok.endswith("/>") and not tok.startswith("</"):
                    stack.append(node)
            else:
                txt = tok.strip()
                if txt:
                    stack[-1].add_child(Tree(txt))
            i += 1
        return root

    pt = parse(pred)
    gt = parse(gt)
    apted = APTED(pt, gt, APTEDConfig())
    dist = apted.compute_edit_distance()
    n1 = len(str(pt))
    n2 = len(str(gt))
    return 1 - dist / max(n1, n2)


def truncate_after_first_closing_table(s: str) -> str:
    if not isinstance(s, str):
        return s
    m = re.search(r"</table\s*>", s, flags=re.IGNORECASE)
    if not m:
        return s
    return s[: m.end()]


def evaluate_and_print(model, ds, use_beam=True, beam_size=3):
    results = []
    for p, toks in tqdm(ds.pairs):
        img = Image.open(p).convert("RGB")
        img = ds.tf(img)
        pred_raw = (
            decode_beam(model, img, ds.stoi, ds.itos, beam_size)
            if use_beam
            else decode_beam(model, img, ds.stoi, ds.itos, 1)
        )
        pred = truncate_after_first_closing_table(pred_raw)
        results.append((p, pred, " ".join(toks)))
    for p, pred, gt in results:
        print(f"IMAGE: {os.path.basename(p)}")
        print(f"PRED : {pred}")
        print(f"GT   : {gt}")
        print("-" * 80)
    scores = [teds_score(pred, gt) for _, pred, gt in results]
    print(f"TEDS Score: {sum(scores)/len(scores):.4f}")


if __name__ == "__main__":
    if MODEL_SAVE.exists():
        ckpt = torch.load(MODEL_SAVE, map_location=DEVICE)
        ds = TableBankDS(SRC_FILE, TGT_FILE, IMAGES_DIR, SUBSET_SIZE, ckpt["itos"], ckpt["stoi"])
        model = Model(ds.vocab).to(DEVICE)
        model.load_state_dict(ckpt["state"])
        model, ds = train(ds, model=model, resume=True)
    else:
        ds = TableBankDS(SRC_FILE, TGT_FILE, IMAGES_DIR, SUBSET_SIZE)
        model, ds = train(ds)
    evaluate_and_print(model, ds, use_beam=True, beam_size=4)


C:\Users\ahmed\AppData\Local\Temp\ipykernel_32404\3321288574.py:358: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  ckpt = torch.load(MODEL_SAVE, map_location=DEVICE)
C:\User

IMAGE: 1809.11052.table_8.png
PRED : <table> <thead> <tr> <td> <td> <td> <td> <td> <td> </tr> </thead> <tbody> <tr> <td> <td> <td> <td> <td> <td> </tr> <tr> <td> <td> <td> <td> <td> <td> </tr> <tr> <td> <td> <td> <td> <td> <td> </tr> <tr> <td> <td> <td> <td> <td> <td> </tr> <tr> <td> <td> <td> <td> <td> <td> </tr> <tr> <td> <td> <td> <td> <td> <td> </tr> </tbody> </table>
GT   : <table>   <thead>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   </thead>   <tbody>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   <tr>   <td>   <td>   <td>   <td>   <td>   <td>   </tr>   </tbody>   </table>
--------------------------------------------------------------------------------
IMAGE: 1509.00500.table_4.png
PRED : <table> <thead> <tr> <td> <td> <td> <td> <td> </tr> </thead> <tbody> <tr> <td> <td> <td> <td> <td

AttributeError: 'Tree' object has no attribute 'add_child'